In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
#data_path = os.path.join('..', 'Data', 'seq_struct_data.csv')
#df = pd.read_csv(data_path)
data_path = os.path.join('..', 'Data', 'seq_data.csv')
df = pd.read_csv(data_path)
#df.columns

In [3]:
def getData(data):
    data = data.reindex(np.random.permutation(df.index))
    data = data.values
    X = data[:,:-2]
    y = data[:,-1]
    #Change the y to a 2dimensionnal 
    ytemp = []
    for i in y:
        if i is 0:
            ytemp.append([1,0])
        else:
            ytemp.append([0,1])
    y = np.array(ytemp)
    
    t = 0.15 #keep 15 percent for testing
    x_train = X[t*len(X):]
    x_test = X[:t*len(X)]
    y_train = y[t*len(y):]
    y_test = y[:t*len(y)]
    return (x_train,y_train,x_test,y_test)
    
xy = getData(df)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:16: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [4]:

#2 is hard coded but it should be the number of unique things in y or xy[1]
numbClasses = 2
numbDim= len(xy[0][0])
x = tf.placeholder("float", shape=[None,numbDim])
y_ = tf.placeholder("float", shape=[None, numbClasses])

W = tf.Variable(tf.zeros([numbDim,numbClasses]))
b = tf.Variable(tf.zeros([numbClasses]))

In [5]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [6]:
import random

for i in range(1000):
    index = random.randint(0,len(xy[0])-50)
    batch = [xy[0][index:index+50],xy[1][index:index+50]]

    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    

In [7]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

print accuracy.eval(feed_dict={x: xy[2], y_: xy[3]})

0.549383


In [8]:
#lets try with two layers. 
